# MinSum reviewed
Min-Sum is an approximation to belief propagation, but we can see it as an implementation of BP.
The usual presentation of goes like this:

There are variable nodes and check nodes, and they send messages between them (check nodes to variable nodes and variable nodes to check nodes, but not check to check or variable to variable !).
After a few rounds of this, the information we want is in the variable nodes (hopefully).

In the classical case we have a message $\vec{m}$ that is encoded (using a generator matrix $G$) into a codeword: $G\cdot\vec{m} = \vec{c}$.

Then we "send" the codeword $\vec{c}$ through a noisy channel, and obtain $\vec{r} = \vec{c} + \vec{n}$, where $\vec{n}$ is a noise vector.

On the "recieve"side, we have a matrix, $H$, called the parity check matrix, and the codewords are in $ker\{H\}$, so $H\cdot \vec{c} = \vec{0}$.

So, unless the noise $\vec{n}$ is also in $ker\{H\}$, we have $H\cdot (\vec{c} + \vec{n}) = \vec{0} + H\cdot \vec{n}$.

The part $H\cdot \vec{n}$ is called the syndrome, and if it is non zero, then we know there is some noise.

In the classical case, the variable nodes are the observed bits (or approximation of them) from a channel, and the check nodes are defined as the XOR equations that the bits in the codeword must satisfy.

Note that. In the classical case, we don't care about the error $\vec{e}$, we only care about the uncorrupt codeword $\vec{c}$.

Actually, in the classical case we don't even care about the uncorrupt copdeword $\vec{c}$, we actually only really care about the message $\vec{m}$ that was encoded into the codeword.

In the quantum case, we ONLY care about the error $\vec{e}$ (in fact if we could decode the "message"it would not be very quantum of us ...)

In [ ]:
from src.minSum import *
from src.polynomialCodes import A1_HX, A1_HZ


def evaluateCode(numberOfTransmissions, seed, errorRange, numberOfIterations, H):
    """
    parameters
    ----------
    numberOfTransmissions : int
        Number of transmissions to simulate at each error probability.
    seed : int
        Seed for the local PRNG.
    errorRange : list of float
        List of error probabilities to simulate.
    numberOfIterations : int
        Number of iterations for the decoder.
    H : ndarray
        Parity check matrix.
    Returns
    ------- 
    berArray : ndarray
        Array of bit error rates for each error probability in errorRange.

    Notes
    -----
    Concurrent futures require the seed to be between 0 and 2**32 -1
    """

    
    
    codewordSize = H.shape[1]
    localPrng = np.random.RandomState(seed)
    decoder = ldpcDecoder(H, syndromeDecoding = True)
    start = 0
    end = 0
    codeword = np.zeros(codewordSize, dtype = LDPC_INT_DATA_TYPE)
    decodedWord = np.zeros(codewordSize, dtype = LDPC_INT_DATA_TYPE)
    berArray = np.zeros(len(errorRange))
    for i,p in zip(range(len(errorRange)), errorRange):
        timeTotal = 0    
        print(f"Error prob {p}, corresponding to i=={i}")
        for k in range(numberOfTransmissions):
            error = localPrng.choice([0,1], size=codewordSize, replace=True, p=[1 - p, p])
            errorModulated = np.where(error == 0, -1.0, 1.0)
            berUncoded = 0
            berDecoded = 0
            start = time.time()
            status, decodedWord, softVector, iterationStoppedAt = decoder.decoderMainLoop(errorModulated, numberOfIterations)
            end = time.time()
            timeTotal += (end - start)
            #print("******** " + str(np.sum(decodedWord == codeword)))
            berDecoded = np.count_nonzero(decodedWord != codeword)
            berArray[i] += berDecoded
        print("Time it took the decoder:")
        print(timeTotal)
        print("And the throughput is:")
        numberOfBits = numberOfTransmissions * codewordSize
        print(numberOfBits / timeTotal)
    return berArray / (numberOfTransmissions * codewordSize)


def evaluateCodeAtSingleTransmission(seed, SNRpoints, messageSize, codewordSize, numberOfIterations, H, G = 'None' ):
    """
    Parameters
    ----------  
    seed : int
        Seed for the local PRNG.
    SNRpoints : list of float
    """

    # Concurrent futures require the seed to be between 0 and 2**32 -1
    #assert (np.dtype(seed) == np.int32)
    assert (seed > 0)
    assert hasattr(SNRpoints, "__len__")
    localPrng = np.random.RandomState(seed)
    decoder = ldpcDecoder(H)
    numberOfSNRpoints = len(SNRpoints)
    
    # init a new berStatistics object to collect statistics
    berStats = common.berStatistics()#np.zeros(numberOfSNRpoints, dtype = LDPC_DECIMAL_DATA_TYPE)
    for s in range(numberOfSNRpoints):
        berUncoded = 0
        berDecoded = 0
        SNR = SNRpoints[s]
        
        #print("*** transmission number " + str(transmission))
        ## loc == mean, scale == standard deviation (AKA sigma).
        if G == 'None':
            # Omer Sella: If G is not given we use the all 0 codeword, do not pass through message generation, do encode using multiplication by G.
            codeword = np.zeros(codewordSize, dtype = LDPC_INT_DATA_TYPE)
        else:
            message = localPrng.randint(0, 2, messageSize, dtype = LDPC_INT_DATA_TYPE)
            codeword = G.dot(message) % 2    
        modulatedCodeword = modulate(codeword, codewordSize)    
        dirtyModulated, sigma, sigmaActual = addAWGN(modulatedCodeword, codewordSize, SNR, LDPC_LOCAL_PRNG) 
        dirtyModulated = copy.copy(modulatedCodeword)
        dirtyModulated[0] = dirtyModulated[0] * -1
        
        
        senseword = slicer(dirtyModulated, codewordSize)
        berUncoded = np.count_nonzero(senseword != codeword)
        start = time.time()
        status, decodedWord, softVector, iterationStoppedAt = decoder.decoderMainLoop(dirtyModulated, numberOfIterations)
        end = time.time()
        #print("******** " + str(np.sum(decodedWord == codeword)))
        berDecoded = np.count_nonzero(decodedWord != codeword)
        berStats.addEntry(SNR, sigma, sigmaActual, berUncoded, berDecoded, iterationStoppedAt, numberOfIterations, status)
        
    return berStats

def constantFunction(const):
    def g(x):
        return const
    return g


def testCodeUsingMultiprocessing(seed, SNRpoints, messageSize, codewordSize, numberOfIterations, numberOfTransmissions, H, method = None, reference = None, G = 'None'):
    bStats = common.berStatistics()
    seeds = LDPC_LOCAL_PRNG.randint(0, LDPC_MAX_SEED, numberOfTransmissions, dtype = LDPC_SEED_DATA_TYPE) 
    
    mesL = [messageSize] * numberOfTransmissions
    cwsL = [codewordSize] * numberOfTransmissions
    itrL = [numberOfIterations] * numberOfTransmissions
    hL = [H] * numberOfTransmissions
    if method == None:
        #Omer Sella: This is a cheasy fix, and only works for the constant function. Need to generalise this.
        #g = constantFunction(reference)
        
        snrL = [SNRpoints] * numberOfTransmissions
        for s in SNRpoints:
            sL = [[s]] * numberOfTransmissions
            start = time.time()
            with concurrent.futures.ProcessPoolExecutor(max_workers=2) as executor:
            # Omer Sella: we are using multiprocessing and distributing over TRANSMISSIONS (which we expect to have O(numberOfTransmissions)) rather than SNR points (which we expect to have O(3)).
                results = executor.map(evaluateCodeAtSingleTransmission, seeds, sL, mesL, cwsL, itrL, hL)
                for r in results:
                    bStats = bStats.union(r)
            end = time.time()
            print(" Time it took the decoder at snr "+ str(s) + " is:")
            print(end-start)
            print("And the throughput is: ")
            print(numberOfTransmissions * codewordSize / (end-start))
    else:
        for s in SNRpoints:
            snrL = [[s]] * numberOfTransmissions
            with concurrent.futures.ProcessPoolExecutor() as executor:
                results = executor.map(evaluateCodeAtSingleTransmission, seeds, snrL, mesL, cwsL, itrL, hL)
            for r in results:
                bStats = bStats.union(r)
            snrAxis, averageSnrAxis, berData, averageNumberOfIterations = bStats.getStats(messageSize)
            print(berData)
            if (berData[0] > reference): 
                print("*** Evaluation failed at " + str(len(bStats.snrAxis)))
                break
    return bStats



def testNearEarth(numOfTransmissions = 50):
    print("*** in test near earth")
    nearEarthParity = fileHandler.readMatrixFromFile(str(projectDir) + '/codeMatrices/nearEarthParity.txt', 1022, 8176, 511, True, False, False)
    #numOfTransmissions = 50
    roi = [3.0, 3.2, 3.4,3.6]#np.arange(3, 3.8, 0.2)
    codewordSize = 8176
    messageSize = 7154
    numOfIterations = 50

    start = time.time()
    
    #bStats = evaluateCode(numOfTransmissions, 460101, roi, messageSize, codewordSize, numOfIterations, nearEarthParity)    
    #for i in range(numOfTransmissions):
    #    bStats = evaluateCodeAtSingleTransmission(460101, roi, messageSize, codewordSize, numOfIterations, nearEarthParity)    
        
    bStats = testCodeUsingMultiprocessing(460101, roi, messageSize, codewordSize, numOfIterations, numOfTransmissions, nearEarthParity)
    end = time.time()
    print('****Time it took for code evaluation == %d' % (end-start))
    print('****Throughput == '+str((8176*len(roi)*numOfTransmissions)/(end-start)) + 'bits per second.')
    #a, b, c, d = bStats.getStats(codewordSize)
    #print("berDecoded " + str(c))
    return bStats


if __name__ == "__main__":
    evaluateCode(50, 7134066, [0.01,0.02],50, A1_HX)




Error prob 0.01, corresponding to i==0


AttributeError: 'checkNode' object has no attribute 'fromChannel'